# Full regional extraction
Here we will extract cells from a variable size continuous polygon.
This will be the first step in doing a full regionalising model.

In [ ]:
#Loading things as usual
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
from drivers.tree_builder import TreeNeo
from drivers.graph_models import TreeNode, Order, Family, graph,Kingdom,Occurrence
from drivers.graph_models import Cell,Mex4km, countObjectsOf
from drivers.graph_models import pickNode
import matplotlib.pyplot as plt
import pandas as pd
import itertools as it
import numpy as np

## Use the ggplot style
plt.style.use('ggplot')

# Ok, the approach will be this
1. Let $p=(x,y)$ in Mexico
2. Create a buffer size list (maybe exponential) not to big.
4. Generate a list of polygons according to this list.
5. Overlay with the mesh (lowest resolution)
6. Extract cells, or cell ids.


In [2]:
from django.contrib.gis.geos import Point, Polygon
xcoord = -99.76
ycoord = 17.55
p = Point(xcoord,ycoord,srid=4326)
radii = np.linspace(0.08,1, 10)
polys = map(lambda r : p.buffer(r),radii)


In [3]:
## Intersect the polygons with the cells to obtain the targets

In [4]:
from mesh.models import initMesh

In [5]:
mexmesh = initMesh(11)

DEBUG Changed MESH TABLE SPACE


In [6]:
intersected_cells = map(lambda poly : mexmesh.objects.filter(cell__intersects=poly),polys)

In [7]:
# Number of selected cells in each buffer
values = map(lambda l : list(l.values_list('id',flat=True)),intersected_cells)

In [8]:
## Change to network database
from traversals.strategies import idsToCells
cells = map(lambda level : list(idsToCells(level)),values)

INFO Compiling Query and asking the Graph Database
INFO Compiling Query and asking the Graph Database
INFO Compiling Query and asking the Graph Database
INFO Compiling Query and asking the Graph Database
INFO Compiling Query and asking the Graph Database
INFO Compiling Query and asking the Graph Database
INFO Compiling Query and asking the Graph Database
INFO Compiling Query and asking the Graph Database
INFO Compiling Query and asking the Graph Database
INFO Compiling Query and asking the Graph Database


In [9]:
### Sumon the LToL
# Following example from Data generator
from drivers.tree_builder import buildTreeNeo
trees = []
for list_of_cells in cells:
    %time trees.append(map(lambda c : buildTreeNeo(c),list_of_cells))


CPU times: user 40 s, sys: 1.12 s, total: 41.1 s
Wall time: 43.9 s
CPU times: user 58.4 s, sys: 1.55 s, total: 60 s
Wall time: 1min 3s
CPU times: user 1min 59s, sys: 3.61 s, total: 2min 3s
Wall time: 2min 11s
CPU times: user 2min 42s, sys: 4.57 s, total: 2min 46s
Wall time: 2min 58s
CPU times: user 3min 37s, sys: 6.34 s, total: 3min 43s
Wall time: 3min 59s
CPU times: user 4min 8s, sys: 6.88 s, total: 4min 14s
Wall time: 4min 32s
CPU times: user 4min 40s, sys: 7.46 s, total: 4min 47s
Wall time: 5min 7s
CPU times: user 4min 59s, sys: 8.36 s, total: 5min 8s
Wall time: 5min 30s
CPU times: user 5min 51s, sys: 9.26 s, total: 6min
Wall time: 6min 25s
CPU times: user 5min 55s, sys: 9.88 s, total: 6min 5s
Wall time: 6min 32s


In [13]:
## Pseudo Presence bursea 
from traversals import strategies as st
## Get all models, environmental variables
from raster_api.models import raster_models_dic

In [14]:
rstmods = raster_models_dic.keys()

In [23]:
%time data = map(lambda cell : st.getEnvironmentalCovariatesFromListOfCells(cell,vars=rstmods),cells)

CPU times: user 1min 40s, sys: 2.17 s, total: 1min 42s
Wall time: 28min 36s


## Let's build the Pseudo presence absence
First, Let be bursera the node we want to model

In [24]:
bursera = pickNode(Family,'Burseracea')

In [25]:
root_node = bursera.getParent().getParent().getParent().getParent().getParent()

In [26]:
t = trees[0][0]

In [27]:
presences = []
for ltree in trees:
    %time presences.append(st.getPresencesForNode(root_node,ltree))
    

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.06 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.07 ms
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.95 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.5 ms
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.72 ms
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.44 ms
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.45 ms
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.43 ms
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 7.71 ms
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 8.68 ms


In [28]:

from ecoregions.models import TerrEcoregions,InegiIV

In [ ]:
%%time
def extractVegetation(list_points):
    """
    Given a list of points it will return a dataframe of the corresponding vegetation 
    type from the TerrEcoregions and INEGI series V.
    To be included in data extraction method, but there's no time for it, there's never time for anything!
    Returns a dataframe.
    """
    points = list_points
    ecoregions = map(lambda p : TerrEcoregions.objects.filter(geom__intersects=p),points)
    ecovalues = map(lambda q : q.values_list('wwf_mhtnum','wwf_mhtnam'),ecoregions)
    inegiregions = map(lambda p : InegiIV.objects.filter(geom__intersects=p),points)
    inegivalues = map(lambda q : q.values_list('covid','name'),inegiregions)
    ecovals = [r.get() if r.exists() else (np.nan,'NaN') for r in ecovalues ]
    inegivals = [r.get() if r.exists() else (np.nan,'NaN') for r in inegivalues ]
    # Compile dataframe
    vegdat = pd.DataFrame(ecovals)
    inegidat = pd.DataFrame(inegivals)
    vegdat.columns = ['vegid','vegname']
    inegidat.columns = ['inegiv5id','inegiv5name']
    results = pd.concat([vegdat,inegidat],axis=1)
    return results

In [29]:
centers_nested = map(lambda c : map(lambda p : p.centroid, c),cells)

In [30]:
from utilities import data_extraction as de

In [31]:
## Calculate tipo de vegetacion
#vegie_types = map(lambda lp : extractVegetation(lp),centers_nested)
from ecoregions.models import TerrEcoregions,InegiIV
vectsel = [[TerrEcoregions,['wwf_mhtnum','wwf_mhtnam'],[np.nan,'NaN']],[InegiIV,['covid','name'],[np.nan,'NaN']]]
vegie_types = map(lambda lp : de.extractSeveralVectorFeatures(lp,vectsel),centers_nested)

INFO extracting info from: <class 'ecoregions.models.TerrEcoregions'> this can take some minutes
INFO extracting info from: <class 'ecoregions.models.InegiIV'> this can take some minutes
INFO extracting info from: <class 'ecoregions.models.TerrEcoregions'> this can take some minutes
INFO extracting info from: <class 'ecoregions.models.InegiIV'> this can take some minutes
INFO extracting info from: <class 'ecoregions.models.TerrEcoregions'> this can take some minutes
INFO extracting info from: <class 'ecoregions.models.InegiIV'> this can take some minutes
INFO extracting info from: <class 'ecoregions.models.TerrEcoregions'> this can take some minutes
INFO extracting info from: <class 'ecoregions.models.InegiIV'> this can take some minutes
INFO extracting info from: <class 'ecoregions.models.TerrEcoregions'> this can take some minutes
INFO extracting info from: <class 'ecoregions.models.InegiIV'> this can take some minutes
INFO extracting info from: <class 'ecoregions.models.TerrEcoregio

In [32]:
datos = zip(presences,data,vegie_types)

In [33]:
## Compile the dataframe and convert to geopandas
dd = map(lambda (p,d,v) : pd.concat([p,d,v],axis=1),datos)
dd = map(lambda d : st.toGeoDataFrame(d,xcoord_name='Longitude',ycoord_name='Latitude'),dd)

In [34]:
## Maybe this was not necessary
#def encode_df(df):
#    df['inegiv5name'] = df['inegiv5name'].apply(lambda x: x.decode('utf8'))
#    return df

#dataframes = map(encode_df,dd)

In [35]:
for i,d in enumerate(dd):
    cad = '/outputs/presence_only_models/data/burseras/%s.csv'%i
    print(cad)
    d.to_csv(cad,encoding='utf8')

/outputs/presence_only_models/data/burseras/0.csv
/outputs/presence_only_models/data/burseras/1.csv
/outputs/presence_only_models/data/burseras/2.csv
/outputs/presence_only_models/data/burseras/3.csv
/outputs/presence_only_models/data/burseras/4.csv
/outputs/presence_only_models/data/burseras/5.csv
/outputs/presence_only_models/data/burseras/6.csv
/outputs/presence_only_models/data/burseras/7.csv
/outputs/presence_only_models/data/burseras/8.csv
/outputs/presence_only_models/data/burseras/9.csv


In [ ]:
d9 = dd[9]

In [ ]:
plt.scatter()
dd[9].plot(column='DistanceToRoadMex_mean')
dd[9].plot(column='Burseraceae',cmap=plt.cm.Greys)

In [ ]:
otro = d.replace('NaN',np.nan)

In [ ]:
otro2 = otro.replace('N.A.',np.nan)

In [ ]:
plt.scatter(otro2.Longitude,otro2.Latitude,c=otro2.Elevation_mean,cmap=plt.cm.Accent,s=30)
#plt.scatter(otro2.Longitude,otro2.Latitude,c=otro2.Precipitation_mean,cmap=plt.cm.Accent,s=30)

#plt.scatter(otro2.Longitude,otro2.Latitude,c=otro2.DistanceToRoadMex_mean,cmap=plt.cm.Accent,s=30)
plt.scatter(otro2.Longitude,otro2.Latitude,c=otro2.Burseraceae,cmap=plt.cm.Greys,s=5,vmin=0.5,vmax=1.0)

In [ ]:
d9.columns

In [ ]:
## This is for plotting the polygons
from geopandas import GeoSeries,GeoDataFrame
from shapely.wkt import loads

load = map(lambda p : loads(p.wkt),polys)
#s = GeoDataFrame(GeoSeries(load))
